<a href="https://colab.research.google.com/github/elephant-xyz/notebook/blob/main/Mining_Seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Seed Mining process

In [ ]:
# @title Step 1: Upload .env

In [19]:
# @title Step 2: Prepare
# @title  {"vertical-output":true}

import csv
import json
import os
from urllib.parse import urlparse, parse_qs

parcel_id = "153323ZZZ000000000200A" # @param {"type":"string"}
address = "EGMONT KEY, ST PETERSBURG, FL 33715" # @param {"type":"string"}
request_method = "GET" # @param {"type":"string"}
url = "https://gis.hcpafl.org/propertysearch/#/parcel/basic/153323ZZZ000000000200A" # @param {"type":"string"}
County = "hillsborough" # @param {"type":"string"}
headers = "" # @param {"type":"string"}
json_body = "" # @param {"type":"string"}

fieldnames = [
    "parcel_id",
    "address",
    "method",
    "url",
    "multiValueQueryString",
    "county",
    "json",
    "headers",
    "source_identifier",
]


def extract_base_url_and_params(url):
    parsed = urlparse(url)
    base_url = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"

    # Collect params from the standard query part
    params = {}
    if parsed.query:
        for k, v in parse_qs(parsed.query, keep_blank_values=True).items():
            params[k] = v[0] if len(v) == 1 else v

    # Include fragment in base_url; also parse ?query that might be inside the fragment
    if parsed.fragment:
        frag_path, _, frag_query = parsed.fragment.partition('?')
        # Keep the path-like part of the fragment in the base URL
        base_url = f"{base_url}#{frag_path}" if frag_path else f"{base_url}#"

        if frag_query:
            for k, v in parse_qs(frag_query, keep_blank_values=True).items():
                if k in params:
                    # Merge with existing values
                    existing = params[k] if isinstance(params[k], list) else [params[k]]
                    merged = existing + v
                    params[k] = merged if len(merged) > 1 else merged[0]
                else:
                    params[k] = v[0] if len(v) == 1 else v

    return base_url, params


def process_input_data():
  base_url, params = extract_base_url_and_params(url)

  row = {
      "parcel_id": parcel_id,
      "address": address,
      "method": request_method,
      "url": base_url,
      # Important: multiValueQueryString must be JSON string with array values
      "multiValueQueryString": json.dumps(params, ensure_ascii=False) if params else "",
      "county": County,
      "json": json_body,
      "headers": headers,
      "source_identifier": parcel_id,
  }



  with open("seed.csv", "w", newline="", encoding="utf-8") as f:
      writer = csv.DictWriter(f, fieldnames=fieldnames)
      writer.writeheader()
      writer.writerow(row)


process_input_data()
print(f"✅ Prepare done for parcel ID {parcel_id}")


✅ Prepare done for parcel ID 153323ZZZ000000000200A


In [20]:
# @title Step 2: Transform
! pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv
import os


def has_submit_errors(path="submit_errors.csv"):
    if not os.path.exists(path):
      return False
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_transform():
    try:
        subprocess.run(
            [
              "npx", "-y", "@elephant-xyz/cli@latest", "transform",
              "--group", "seed",
              "--input-csv", "seed.csv",
              "--output-zip", "transformed_seed.zip",
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Transform failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        # seed_group_cid, html_link = get_seed_cid_and_html_link()

        print("✅ Transform done\n")
        # print(f"Seed group CID: {seed_group_cid}\n")
        # print(f"HTML link: {html_link}")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_transform()


!find output -type f -name "seed_data_group.json" -execdir mv seed_data_group.json bafkreif7ywbjxu3s6jfi6ginvmsufeux3cd5eujuivg2y7tmqt2qk4rsoe.json \;

✅ Transform done



In [22]:
# @title Step 3: Validate
!pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_validate():
    try:
        subprocess.run(
            ["npx", "-y", "@elephant-xyz/cli", "validate", "transformed_seed.zip"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Validate failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        # seed_group_cid, html_link = get_seed_cid_and_html_link()
        print("✅ Validate done\n")
        # print(f"Seed group CID: {seed_group_cid}\n")
        # print(f"HTML link: {html_link}")

    except subprocess.CalledProcessError as e:
        # обробка помилок виконання команди
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_validate()


✅ Validate done



In [ ]:
# @title Step 4: Hash
!pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def get_seed_cid(path="seed-results.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row["propertyCid"]


def run_hash():
    try:
        subprocess.run(
            [
                "npx", "-y", "@elephant-xyz/cli",
                "hash", "transformed_seed.zip",
                "--output-zip", "hashed-data.zip",
                "--output-csv", "seed-results.csv",
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Validate failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        seed_group_cid = get_seed_cid()
        print("✅ Hash done\n")
        print(f"Seed group CID: {seed_group_cid}\n")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_hash()



✅ Hash done

Seed group CID: bafkreigmp55sariml2ejwibavy2py3gfxec43vd5qhtxaiq57ty6clnmgu



In [ ]:
# @title Step 5: Upload
! pip3 install python-dotenv requests -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv

import requests


def get_seed_info(path="seed-results.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def count_uploaded_files(output_dir="output"):
    """
    Count all files recursively in the output directory and its subdirectories.
    """
    if not os.path.exists(output_dir):
        return 0

    file_count = 0
    for root, dirs, files in os.walk(output_dir):
        file_count += len(files)
    return file_count


def collect_data_ipfs_links(data_cid):
  seed_data_link = f"https://ipfs.io/ipfs/{data_cid}"

  seed_data = requests.get(seed_data_link).json()

  property_seed_cid = seed_data["relationships"]["property_seed"]["/"]
  property_seed_link = f"https://ipfs.io/ipfs/{property_seed_cid}"

  property_seed_data = requests.get(property_seed_link).json()

  property_cid, address_cid = property_seed_data["from"]["/"], property_seed_data["to"]["/"]

  property_link = f"https://ipfs.io/ipfs/{property_cid}"
  address_link = f"https://ipfs.io/ipfs/{address_cid}"

  return seed_data_link, property_seed_link, property_link, address_link


def run_validate_and_upload():
    try:
        subprocess.run(
            ["npx", "-y", "@elephant-xyz/cli", "upload", "hashed-data.zip", "--output-csv", "seed-results.csv"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Validate failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        seed_info = get_seed_info()
        seed_group_cid, data_cid, html_link = seed_info["dataGroupCid"], seed_info["dataCid"], seed_info["htmlLink"]

        files_uploaded = count_uploaded_files()

        data_ipfs_links = collect_data_ipfs_links(data_cid)
        seed_group_link, property_seed_link, property_link, address_link = data_ipfs_links

        print("✅ Upload done\n")
        print(f"{files_uploaded} files uploaded\n")

        print(f"Seed group CID: {seed_group_cid}\n")
        print(f"HTML link: {html_link}\n")

        print(f"Seed group IPFS link: {seed_group_link}")
        print(f"relationship IPFS link: {property_seed_link}")
        print(f"property_seed IPFS link: {property_link}")
        print(f"unnormalized_address IPFS link: {address_link}")


        print("✅ Upload done\n")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_validate_and_upload()


✅ Upload done

4 files uploaded

Seed group CID: bafkreif4dh7lh3ik4u726sqkjybcuqztqmoxg4m4dybzafcbqa7t4pes64

HTML link: https://ipfs.io/ipfs/bafybeigscsggt6bq3kww2bpwfyyokkdyikbdwbadyrdewnmuedricaw4ri

Seed group IPFS link: https://ipfs.io/ipfs/bafkreigmp55sariml2ejwibavy2py3gfxec43vd5qhtxaiq57ty6clnmgu
relationship IPFS link: https://ipfs.io/ipfs/bafkreigacxivvzfdbwg7lexvu67tgkl76gcfzcql2vlscmbyqm4t43fcpe
property_seed IPFS link: https://ipfs.io/ipfs/bafkreiawsrtadbrmno7guioh75ivlgmiscbvocsoz4voc5zl2n62c3va4m
unnormalized_address IPFS link: https://ipfs.io/ipfs/bafkreig45uumuublszfnm2w267jbxsurrhuivd2mnmilact4fyutpal4de
✅ Upload done



In [ ]:
# @title Step 6: Submit

! pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def get_transaction_hash(path="transaction-status.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row["transactionHash"]


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_submit_to_contract():
    try:
        subprocess.run(
            [
                "npx", "-y", "@elephant-xyz/cli", "submit-to-contract", "seed-results.csv",
                "--from-address", "0xefAd08946612A15d5De8D4Db7fc03556b6424075",
                "--api-key", "f7e18cf6-5d07-4e4a-ae23-f27b812614e6",
                "--domain", "oracles-69c46050.staircaseapi.com",
                "--oracle-key-id", "7ad26e0b-67c9-4c2f-95a2-2792c7db5ac7",
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Submit failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        transaction_hash = get_transaction_hash()
        transaction_link = f"https://polygonscan.com/tx/{transaction_hash}"

        print("✅ Submit done\n")
        print(f"Transaction link: {transaction_link}")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_submit_to_contract()


✅ Submit done

Transaction link: https://polygonscan.com/tx/0x07b662cf78ce10fda3409efcf0b1cec2ec5473709fc4fc4cc5fc3e8228941532


In [ ]:
# @title Step 6: Construct seed_output.zip and Download it
!mkdir -p seed_output
!cp ./output/*/property_seed.json ./output/*/unnormalized_address.json ./seed-results.csv ./seed.csv seed_output/
!zip -r seed_output.zip seed_output


import os; from google.colab import files; (files.download('seed_output.zip'), print("✅ File was downloaded successfully"))[1] if os.path.exists('seed_output.zip') else print("❌ File not found")


  adding: seed_output/ (stored 0%)
  adding: seed_output/property_seed.json (deflated 38%)
  adding: seed_output/seed.csv (deflated 24%)
  adding: seed_output/unnormalized_address.json (deflated 31%)
  adding: seed_output/seed-results.csv (deflated 43%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ File was downloaded successfully
